# baseline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.quantization import QuantStub, DeQuantStub, fuse_modules, prepare_qat, convert
import torchvision
import torchvision.transforms as transforms
import os
import time
from tqdm import tqdm

Using device: cuda
Files already downloaded and verified


Epoch 2/18:   0%|          | 0/49 [00:00<?, ?batch/s]

Epoch 1, Loss: 1.3422739505767822


Epoch 2/18: 100%|██████████| 49/49 [00:14<00:00,  3.36batch/s]


Epoch 2, Loss: 1.1376851797103882


...

Epoch 17/18: 100%|██████████| 49/49 [00:15<00:00,  3.14batch/s]


Epoch 17, Loss: 0.05444111302495003


Epoch 18/18: 100%|██████████| 49/49 [00:14<00:00,  3.31batch/s]


Epoch 18, Loss: 0.05702977627515793
Original Model Training Time: 137.36 seconds
Files already downloaded and verified


<ipython-input-10-242f76e99880>:167: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("original_model.pt")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Model
class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()

        # First Conv Block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second Conv Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Thrid Conv Block
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU()
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.relu6 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 4th Conv Block
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.relu7 = nn.ReLU()
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(512)
        self.relu8 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # FC Layer
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

        # Quantization Stubs
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def forward(self, x):
        # Start quantization
        x = self.quant(x)

        # First Conv Block
        x = self.pool1(self.relu2(self.bn2(self.conv2(self.relu1(self.bn1(self.conv1(x)))))))

        # Second Conv Block
        x = self.pool2(self.relu4(self.bn4(self.conv4(self.relu3(self.bn3(self.conv3(x)))))))

        # Third Conv Block
        x = self.pool3(self.relu6(self.bn6(self.conv6(self.relu5(self.bn5(self.conv5(x)))))))

        # 4th Conv Block
        x = self.pool4(self.relu8(self.bn8(self.conv8(self.relu7(self.bn7(self.conv7(x)))))))

        # Flatten
        x = x.reshape(x.size(0), -1)

        # FC Layer
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)

        # Dequantization
        x = self.dequant(x)
        return x

    def fuse_model(self):
        # Merge BatchNorm and ReLU
        fuse_modules(self, [
            ['conv1', 'bn1', 'relu1'],
            ['conv2', 'bn2', 'relu2'],
            ['conv3', 'bn3', 'relu3'],
            ['conv4', 'bn4', 'relu4'],
            ['conv5', 'bn5', 'relu5'],
            ['conv6', 'bn6', 'relu6'],
            ['conv7', 'bn7', 'relu7'],
            ['conv8', 'bn8', 'relu8']
        ], inplace=True)


In [ ]:
# Instantiate the model
model = ComplexCNN()

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)

# Model, Loss, Optimizer
model = ComplexCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [ ]:
# Pre-QAT Training
model.train()

start_time = time.time()
for epoch in range(18):
    for data, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/18", unit="batch"):
        data, target = data.to(device), target.to(device)  # 데이터를 GPU로 이동
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
end_time = time.time()
print(f"Original Model Training Time: {(end_time - start_time) / 2:.2f} seconds")

torch.save(model, "original_model.pt")


In [ ]:
# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for inference
        for data, target in test_loader:
            data, target = data.to('cpu'), target.to('cpu')
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [ ]:
# Dataset and DataLoader for Evaluation
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False)

# Load the original model
model = torch.load("original_model.pt")
model.to('cpu')

# Evaluate the model
accuracy = evaluate_model(model, test_loader)
print(f"Model Accuracy: {accuracy:.2f}%")

# PTQ, QAT 적용 및 평가

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.quantization import QuantStub, DeQuantStub, fuse_modules, prepare_qat, convert, quantize_dynamic
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm
import pandas as pd
import contextlib
import random
from collections import defaultdict

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
Starting Quantization...


<ipython-input-12-99eaf9410049>:154: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  original_model = torch.load("original_model.pt", map_location=device)
<ipython-input-12-99

Calibration Time: 83.01 seconds


<ipython-input-12-99eaf9410049>:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qat_model = torch.load("original_model.pt", map_location=device)


Epoch 1:


...

Epoch 3:


QAT Fine-tuning: 100%|██████████| 49/49 [00:19<00:00,  2.54batch/s]


Quantization completed. Starting evaluation iterations...
Iteration 1/5
Iteration 1 ptq model


<ipython-input-12-99eaf9410049>:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  original_model = torch.load("original_model.pt", map_location='cpu')


Inference time: 47.834423303604126, Accuracy: 75.08
Iteration 1 qat model
Inference time: 46.7501118183136, Accuracy: 85.98
Iteration 1 original model
Inference time: 111.24269914627075, Accuracy: 78.99
Iteration 2/5
Iteration 2 ptq model
Inference time: 45.49363327026367, Accuracy: 75.08
Iteration 2 qat model
Inference time: 48.4896445274353, Accuracy: 85.98
Iteration 2 original model
Inference time: 111.10444259643555, Accuracy: 78.99
Iteration 3/5
Iteration 3 ptq model
Inference time: 46.862894773483276, Accuracy: 75.08
Iteration 3 qat model
Inference time: 46.61470437049866, Accuracy: 85.98
Iteration 3 original model
Inference time: 113.87418723106384, Accuracy: 78.99
Iteration 4/5
Iteration 4 ptq model
Inference time: 48.674333810806274, Accuracy: 75.08
Iteration 4 qat model
Inference time: 47.91838598251343, Accuracy: 85.98
Iteration 4 original model
Inference time: 117.29656553268433, Accuracy: 78.99
Iteration 5/5
Iteration 5 ptq model
Inference time: 46.46167063713074, Accuracy

In [ ]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



In [ ]:
# Model
class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()

        # First Conv Block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second Conv Block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Thrid Conv Block
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU()
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.relu6 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 4th Conv Block
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(512)
        self.relu7 = nn.ReLU()
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(512)
        self.relu8 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # FC Layer
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

        # Quantization Stubs
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def forward(self, x):
        # Start quantization
        x = self.quant(x)

        # First Conv Block
        x = self.pool1(self.relu2(self.bn2(self.conv2(self.relu1(self.bn1(self.conv1(x)))))))

        # Second Conv Block
        x = self.pool2(self.relu4(self.bn4(self.conv4(self.relu3(self.bn3(self.conv3(x)))))))

        # Third Conv Block
        x = self.pool3(self.relu6(self.bn6(self.conv6(self.relu5(self.bn5(self.conv5(x)))))))

        # 4th Conv Block
        x = self.pool4(self.relu8(self.bn8(self.conv8(self.relu7(self.bn7(self.conv7(x)))))))

        # Flatten
        x = x.reshape(x.size(0), -1)

        # FC Layer
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)

        # Dequantization
        x = self.dequant(x)
        return x

    def fuse_model(self):
        # Merge BatchNorm and ReLU
        fuse_modules(self, [
            ['conv1', 'bn1', 'relu1'],
            ['conv2', 'bn2', 'relu2'],
            ['conv3', 'bn3', 'relu3'],
            ['conv4', 'bn4', 'relu4'],
            ['conv5', 'bn5', 'relu5'],
            ['conv6', 'bn6', 'relu6'],
            ['conv7', 'bn7', 'relu7'],
            ['conv8', 'bn8', 'relu8']
        ], inplace=True)



In [ ]:
# Instantiate the model
model = ComplexCNN().to(device)
model.eval()

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=False)



In [ ]:
# Function to measure inference time and accuracy
def measure_performance(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    start_time = time.time()

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to('cpu'), target.to('cpu')
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    end_time = time.time()
    inference_time = end_time - start_time
    accuracy = 100 * correct / total

    print(f'Inference time: {inference_time}, Accuracy: {accuracy}')
    return inference_time, accuracy


In [ ]:
# Perform Quantization
print("Starting Quantization...")

# Load the original pre-trained model
original_model = torch.load("original_model.pt", map_location=device)

# PTQ: Post-Training Quantization
ptq_model = torch.load("original_model.pt", map_location='cpu')



In [ ]:
# Calibration (Static Quantization)
ptq_model.eval()
ptq_model.fuse_model()  # Fuse layers
ptq_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(ptq_model, inplace=True)

def create_balanced_subset(dataset, num_samples_per_class):
    class_indices = defaultdict(list)

    for idx, (_, label) in enumerate(dataset):
        class_indices[label].append(idx)

    selected_indices = []
    for cls, indices in class_indices.items():
        selected_indices.extend(random.sample(indices, num_samples_per_class))

    subset = torch.utils.data.Subset(dataset, selected_indices)
    return subset

start_calibration = time.time()
calibration_subset = create_balanced_subset(train_dataset, num_samples_per_class=500)
calibration_loader = torch.utils.data.DataLoader(calibration_subset, batch_size=1024, shuffle=False)



In [ ]:
# Use the test dataset for calibration
with torch.no_grad():
    for data, _ in tqdm(calibration_loader, desc='PTQ Calibration', unit='batch'):
        ptq_model(data)
end_calibration = time.time()
print(f'Calibration Time: {end_calibration - start_calibration:.2f} seconds')

torch.quantization.convert(ptq_model, inplace=True)



In [ ]:
# QAT: Quantization-Aware Training
qat_model = torch.load("original_model.pt", map_location=device)
qat_model.eval()
qat_model.fuse_model()  # Fuse the layers before training
qat_model.train()  # Switch back to train mode for QAT

qat_qconfig = torch.quantization.QConfig(
    activation=torch.quantization.FakeQuantize.with_args(
        observer=torch.quantization.MovingAverageMinMaxObserver,
        quant_min=0,
        quant_max=255,
        dtype=torch.quint8,
        qscheme=torch.per_tensor_affine
    ),
    weight=torch.quantization.FakeQuantize.with_args(
        observer=torch.quantization.MovingAverageMinMaxObserver,
        quant_min=-128,
        quant_max=127,
        dtype=torch.qint8,
        qscheme=torch.per_tensor_affine
    )
)

qat_model.qconfig = qat_qconfig
torch.quantization.prepare_qat(qat_model, inplace=True)

optimizer = optim.Adam(qat_model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# 2 epoch fine-tuning for QAT
for i in range(3):
    print(f'Epoch {i+1}:')
    for data, target in tqdm(train_loader, desc="QAT Fine-tuning", unit="batch"):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = qat_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

qat_model.eval()
qat_model.to('cpu')  # Move to CPU before quantization
convert(qat_model, inplace=True)

print("Quantization completed. Starting evaluation iterations...")



In [ ]:
# Reload the original pre-trained model for each iteration
original_model = torch.load("original_model.pt", map_location='cpu')
original_model.eval()



In [ ]:
# Evaluation Iterations
results = []
for i in range(5):
    print(f"Iteration {i+1}/5")

    print(f'Iteration {i+1} ptq model')
    ptq_time, ptq_accuracy = measure_performance(ptq_model, test_loader)
    print(f'Iteration {i+1} qat model')
    qat_time, qat_accuracy = measure_performance(qat_model, test_loader)
    print(f'Iteration {i+1} original model')
    original_time, original_accuracy = measure_performance(original_model, test_loader)

    results.append({
        "Iteration": i + 1,
        "Original_Inference_Time": original_time,
        "Original_Accuracy": original_accuracy,
        "PTQ_Inference_Time": ptq_time,
        "PTQ_Accuracy": ptq_accuracy,
        "QAT_Inference_Time": qat_time,
        "QAT_Accuracy": qat_accuracy,
    })


In [ ]:
# Convert results to DataFrame and compute statistics
df = pd.DataFrame(results)
df['Original_Inference_Time_Mean'] = df['Original_Inference_Time'].mean()
df['Original_Accuracy_Mean'] = df['Original_Accuracy'].mean()
df['PTQ_Inference_Time_Mean'] = df['PTQ_Inference_Time'].mean()
df['PTQ_Accuracy_Mean'] = df['PTQ_Accuracy'].mean()
df['QAT_Inference_Time_Mean'] = df['QAT_Inference_Time'].mean()
df['QAT_Accuracy_Mean'] = df['QAT_Accuracy'].mean()

df['Original_Inference_Time_Max'] = df['Original_Inference_Time'].max()
df['Original_Accuracy_Max'] = df['Original_Accuracy'].max()
df['PTQ_Inference_Time_Max'] = df['PTQ_Inference_Time'].max()
df['PTQ_Accuracy_Max'] = df['PTQ_Accuracy'].max()
df['QAT_Inference_Time_Max'] = df['QAT_Inference_Time'].max()
df['QAT_Accuracy_Max'] = df['QAT_Accuracy'].max()

df['Original_Inference_Time_Min'] = df['Original_Inference_Time'].min()
df['Original_Accuracy_Min'] = df['Original_Accuracy'].min()
df['PTQ_Inference_Time_Min'] = df['PTQ_Inference_Time'].min()
df['PTQ_Accuracy_Min'] = df['PTQ_Accuracy'].min()
df['QAT_Inference_Time_Min'] = df['QAT_Inference_Time'].min()
#df['QAT_Accuracy_Min'] = df['QAT_Accuracy_Min'].min()

In [ ]:
# Save to CSV
df.to_csv("quantization_performance.csv", index=False)
print("Results saved to quantization_performance.csv")